<a href="https://colab.research.google.com/github/KellemAzevedo/Matplotlib-Seaborn-PySpark/blob/main/PySpark_pr%C3%A1ticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações iniciais

## Instal

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

# Criando ambiente virtual do PySpark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# Criando a sessão PySpark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável
spark

# Lendo um arquivo a partir do disco

In [ ]:
df = spark.read.csv('/content/Estatística Descritiva - Organização Gráfica.csv',header=True)

In [ ]:
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|Matrícula|    Nome|   Cidade|Estado|  País|Idade|Departamento|      Cargo|Salário (R$)|Escolaridade|Nota |
+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|       25000|    Superior|    8|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|       12000|    Superior|    6|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|       28000|         MBA|    9|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|       40000|    Mestrado|    7|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|       23000|    Superior|    5|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|       11000|    Superior|    9|
|        7| Eduardo|  Extrem

In [ ]:
df.printSchema()

root
 |-- Matrícula: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País: string (nullable = true)
 |-- Idade: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Salário (R$): string (nullable = true)
 |-- Escolaridade: string (nullable = true)
 |-- Nota : string (nullable = true)



# Criando um arquivo a partir do zero

In [ ]:
# Criando uma tabela
from pyspark.sql.types import *
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType # importando estrutura da tabela (structType)
schema = StructType([ \
    StructField('nome',StringType(),True), \
    StructField('idade',IntegerType(),True), \
    StructField('altura',DoubleType(),True), \
    StructField('peso', DoubleType(), True), \
  ])

In [ ]:
# Criando os dados em Python para inserir na tabela em PySpark
'''
Criação do objeto (nível Python) que contém as tuplas com os regitros que farão
parte da tabela
'''
dados = [('Douglas',45,1.85,70.),
    ('Daniela',7,1.23,22.),
    ('Pedro',65,1.75,87.),
    ('Maria',64,1.67,64.),
    ('Eduardo',37,1.82,96.),
    ('Ester',37,1.73,68.)
  ]

In [ ]:
# Trazendo para o ambiente pyspark
df = spark.createDataFrame(schema=schema, data=dados)
df.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|Daniela|    7|  1.23|22.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
|Eduardo|   37|  1.82|96.0|
|  Ester|   37|  1.73|68.0|
+-------+-----+------+----+



# Filtros com AND e OR

In [ ]:
# Filtrar idades maior que 40 e peso maior que 70
filtro01 = df.filter((df['idade']>=40)&(df['peso']>=70)) # em pyspark o operador lógico and deve ser usado pelo símbolo &
filtro01.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|  Pedro|   65|  1.75|87.0|
+-------+-----+------+----+



In [ ]:
# Filtrar idade maior que 40 ou peso maior que 50
filtro02 = df.filter((df['idade']>=40)|(df['peso']>=50)) # em pyspark o operador lógico ou deve ser usado pelo símbolo |
filtro02.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
|Eduardo|   37|  1.82|96.0|
|  Ester|   37|  1.73|68.0|
+-------+-----+------+----+



In [ ]:
# Filtrar idade maior que 40 e peso maior que 50, porém com outra notação
filtro03 = df.filter('idade >= 40 AND peso >= 50')
filtro03.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
+-------+-----+------+----+



In [ ]:
# Filtrar idade maior que 40 ou peso maior que 50
filtro04 = df.filter('idade >= 40 OR peso >= 50')
filtro04.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
|Eduardo|   37|  1.82|96.0|
|  Ester|   37|  1.73|68.0|
+-------+-----+------+----+



In [ ]:
# Filtrar as linhas em que nome NÃO é Douglas
filtro05 = df.filter(~(df['nome'] == 'Douglas')) # Filtro com operador lógico simbólico de negação
filtro05.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Daniela|    7|  1.23|22.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
|Eduardo|   37|  1.82|96.0|
|  Ester|   37|  1.73|68.0|
+-------+-----+------+----+



# Filtro com funções

In [ ]:
from pyspark.sql.functions import *

In [ ]:
# A função col permite que use outra função como parâmetro, nesse caso, startswith
# 'Col' significa coluna
filtro06 = df.filter(col('nome').startswith('D'))
filtro06.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|Daniela|    7|  1.23|22.0|
+-------+-----+------+----+



In [ ]:
idade = df.select('idade')
idade.show()

+-----+
|idade|
+-----+
|   45|
|    7|
|   65|
|   64|
|   37|
|   37|
+-----+



## Mostrar coluna

In [ ]:
# selecionando coluna única e armazenando em um dataframe do pyspark
idade = df.select('idade')
idade.show()

+-----+
|idade|
+-----+
|   45|
|    7|
|   65|
|   64|
|   37|
|   37|
+-----+



In [ ]:
# selecionando mais de uma coluna e as armazenando em um dataframe do pyspark
filtro07 = df.select('altura','peso')
filtro07.show()

+------+----+
|altura|peso|
+------+----+
|  1.85|70.0|
|  1.23|22.0|
|  1.75|87.0|
|  1.67|64.0|
|  1.82|96.0|
|  1.73|68.0|
+------+----+



## Renomear

In [ ]:
# Renomear a coluna peso para massa
df = df.withColumnRenamed('peso','massa')
df.show()

+-------+-----+------+-----+
|   nome|idade|altura|massa|
+-------+-----+------+-----+
|Douglas|   45|  1.85| 70.0|
|Daniela|    7|  1.23| 22.0|
|  Pedro|   65|  1.75| 87.0|
|  Maria|   64|  1.67| 64.0|
|Eduardo|   37|  1.82| 96.0|
|  Ester|   37|  1.73| 68.0|
+-------+-----+------+-----+



## Criando nova coluna

In [ ]:
# Criando uma nova coluna
df = df.withColumn('imc',df['massa']/df['altura']**2)
df.show()

+-------+-----+------+-----+------------------+
|   nome|idade|altura|massa|               imc|
+-------+-----+------+-----+------------------+
|Douglas|   45|  1.85| 70.0| 20.45288531775018|
|Daniela|    7|  1.23| 22.0|14.541608830722454|
|  Pedro|   65|  1.75| 87.0|28.408163265306122|
|  Maria|   64|  1.67| 64.0|22.948115744558788|
|Eduardo|   37|  1.82| 96.0|28.982007003985025|
|  Ester|   37|  1.73| 68.0|22.720438370810918|
+-------+-----+------+-----+------------------+



In [ ]:
# Escolher uma determina linha por seu pseudo-índice, isso leva de volta para o Python
tio = df.collect()[0]
print(tio)

Row(nome='Douglas', idade=45, altura=1.85, massa=70.0, imc=20.45288531775018)


In [ ]:
# Escolher um único dado de uma linha. Por ex.: Douglas.
doug = df.collect()[0][0]
print(doug)

Douglas


## Adicionando uma nova linha

In [ ]:
# Criando uma nova linha
elemento = [('Tobias',18,1.77,80.,25.535446)]
nova_linha = spark.createDataFrame(elemento)
nova_linha.show()

+------+---+----+----+---------+
|    _1| _2|  _3|  _4|       _5|
+------+---+----+----+---------+
|Tobias| 18|1.77|80.0|25.535446|
+------+---+----+----+---------+



In [ ]:
# Adicionando ao dataframe
df = df.union(nova_linha)
df.show()

+-------+-----+------+-----+------------------+
|   nome|idade|altura|massa|               imc|
+-------+-----+------+-----+------------------+
|Douglas|   45|  1.85| 70.0| 20.45288531775018|
|Daniela|    7|  1.23| 22.0|14.541608830722454|
|  Pedro|   65|  1.75| 87.0|28.408163265306122|
|  Maria|   64|  1.67| 64.0|22.948115744558788|
|Eduardo|   37|  1.82| 96.0|28.982007003985025|
|  Ester|   37|  1.73| 68.0|22.720438370810918|
| Tobias|   18|  1.77| 80.0|         25.535446|
+-------+-----+------+-----+------------------+



# Medidas descritivas

In [ ]:
# Calcular as principais medidas descritivas do df
df.describe().show()

+-------+-------+------------------+-------------------+------------------+------------------+
|summary|   nome|             idade|             altura|             massa|               imc|
+-------+-------+------------------+-------------------+------------------+------------------+
|  count|      7|                 7|                  7|                 7|                 7|
|   mean|   null|              39.0| 1.6885714285714286| 69.57142857142857|23.369809219019068|
| stddev|   null|21.641010450839243|0.21058874387682508|23.831751529259886| 4.976893829941699|
|    min|Daniela|                 7|               1.23|              22.0|14.541608830722454|
|    max| Tobias|                65|               1.85|              96.0|28.982007003985025|
+-------+-------+------------------+-------------------+------------------+------------------+



## Minúscculo

In [ ]:
#Criar uma nova coluna com os nomes só com letras em caixa baixa
df = df.withColumn('minu',lower(col('nome')))
df.show()

+-------+-----+------+-----+------------------+-------+
|   nome|idade|altura|massa|               imc|   minu|
+-------+-----+------+-----+------------------+-------+
|Douglas|   45|  1.85| 70.0| 20.45288531775018|douglas|
|Daniela|    7|  1.23| 22.0|14.541608830722454|daniela|
|  Pedro|   65|  1.75| 87.0|28.408163265306122|  pedro|
|  Maria|   64|  1.67| 64.0|22.948115744558788|  maria|
|Eduardo|   37|  1.82| 96.0|28.982007003985025|eduardo|
|  Ester|   37|  1.73| 68.0|22.720438370810918|  ester|
| Tobias|   18|  1.77| 80.0|         25.535446| tobias|
+-------+-----+------+-----+------------------+-------+



In [ ]:
# Criar uma nova coluna com os nomes só com letras em caixa alta
df = df.withColumn('maiu',upper(col('nome')))
df.show()

+-------+-----+------+-----+------------------+-------+-------+
|   nome|idade|altura|massa|               imc|   minu|   maiu|
+-------+-----+------+-----+------------------+-------+-------+
|Douglas|   45|  1.85| 70.0| 20.45288531775018|douglas|DOUGLAS|
|Daniela|    7|  1.23| 22.0|14.541608830722454|daniela|DANIELA|
|  Pedro|   65|  1.75| 87.0|28.408163265306122|  pedro|  PEDRO|
|  Maria|   64|  1.67| 64.0|22.948115744558788|  maria|  MARIA|
|Eduardo|   37|  1.82| 96.0|28.982007003985025|eduardo|EDUARDO|
|  Ester|   37|  1.73| 68.0|22.720438370810918|  ester|  ESTER|
| Tobias|   18|  1.77| 80.0|         25.535446| tobias| TOBIAS|
+-------+-----+------+-----+------------------+-------+-------+



In [ ]:
# Descobrir o menor valor da coluna idade
idade_min = df.agg({'idade':'min'}) # como resultado teremos um df spark com uma linha e uma coluna
print(idade_min)

+----------+
|min(idade)|
+----------+
|         7|
+----------+



In [ ]:
# Descobrir o maior valor da coluna idade
idade_max = df.agg({'idade':'max'}) # como resultado teremos um df spark com uma linha e uma coluna
print(idade_max)

+----------+
|max(idade)|
+----------+
|        65|
+----------+



In [ ]:
# Descobrir o valor médio da coluna idade
idade_media = df.agg({'idade':'avg'}) # como resultado teremos um df spark com uma linha e uma coluna
print(idade_media)

+----------+
|avg(idade)|
+----------+
|      39.0|
+----------+



# Ordenação

In [ ]:
# Ordenar de forma crescente por nome
df.orderBy('nome')

nome,idade,altura,massa,imc,minu,maiu
Daniela,7,1.23,22.0,14.541608830722454,daniela,DANIELA
Douglas,45,1.85,70.0,20.45288531775018,douglas,DOUGLAS
Eduardo,37,1.82,96.0,28.982007003985025,eduardo,EDUARDO
Ester,37,1.73,68.0,22.720438370810918,ester,ESTER
Maria,64,1.67,64.0,22.948115744558788,maria,MARIA
Pedro,65,1.75,87.0,28.408163265306122,pedro,PEDRO
Tobias,18,1.77,80.0,25.535446,tobias,TOBIAS


In [ ]:
# Ordenar de forma decrescente por nome
df.orderBy(df['nome'].desc())

Matrícula,Nome,Cidade,Estado,País,Idade,Departamento,Cargo,Salário (R$),Escolaridade,Nota
5,Sandra,Atibaia,SP,Brasil,28,Produção,Analista,23000,Superior,5
9,Pedro,Extrema,MG,Brasil,30,Marketing,Analista,13000,Superior,1
10,Maria,Extrema,MG,Brasil,40,Produção,Analista,12000,MBA,7
3,Luiza,Santos,SP,Brasil,38,Finanças,Gerente,28000,MBA,9
1,Lucas,Atibaia,SP,Brasil,35,Compras,Gerente,25000,Superior,8
4,Fernando,Atibaia,SP,Brasil,36,Marketing,Diretor,40000,Mestrado,7
8,Ester,Itapeva,MG,Brasil,29,Compras,Analista,10000,Superior,2
7,Eduardo,Extrema,MG,Brasil,30,Marketing,Gerente,12000,MBA,4
6,Douglas,Bragança,SP,Brasil,29,Finanças,Analista,11000,Superior,9
2,Ana,São Paulo,SP,Brasil,29,Vendas,Coordenador,12000,Superior,6


In [ ]:
'''
Agora imagine que você deseja agrupar por idade
'''
df.groupBy('idade').agg({'idade':'count'})

idade,count(idade)
29,3
30,2
28,1
35,1
38,1
40,1
36,1


# Exercícios

In [ ]:
df = spark.read.csv('/content/Estatística Descritiva - Organização Gráfica.csv',header=True)

In [ ]:
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|Matrícula|    Nome|   Cidade|Estado|  País|Idade|Departamento|      Cargo|Salário (R$)|Escolaridade|Nota |
+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|       25000|    Superior|    8|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|       12000|    Superior|    6|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|       28000|         MBA|    9|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|       40000|    Mestrado|    7|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|       23000|    Superior|    5|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|       11000|    Superior|    9|
|        7| Eduardo|  Extrem

In [ ]:
df.printSchema()

root
 |-- Matrícula: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País: string (nullable = true)
 |-- Idade: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Salário (R$): string (nullable = true)
 |-- Escolaridade: string (nullable = true)
 |-- Nota : string (nullable = true)



In [ ]:
'''
Neste bloco vamos converter idade em inteiro, matrículaem inteiro, salári em
double e nota em Double
'''
from pyspark.sql.types import *
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
df = df \
  .withColumn("Matrícula" ,
              df["Matrícula"]
              .cast(IntegerType()))   \
  .withColumn("Idade",
              df["Idade"]
              .cast(IntegerType()))    \
  .withColumn("Salário (R$)"  ,
              df["Salário (R$)"]
              .cast(DoubleType()))    \
  .withColumn("Nota "  ,
              df["Nota "]
              .cast(DoubleType()))
df.printSchema()

root
 |-- Matrícula: integer (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Salário (R$): double (nullable = true)
 |-- Escolaridade: string (nullable = true)
 |-- Nota : double (nullable = true)



In [ ]:
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|Matrícula|    Nome|   Cidade|Estado|  País|Idade|Departamento|      Cargo|Salário (R$)|Escolaridade|Nota |
+---------+--------+---------+------+------+-----+------------+-----------+------------+------------+-----+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|     25000.0|    Superior|  8.0|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|     12000.0|    Superior|  6.0|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|     28000.0|         MBA|  9.0|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|     40000.0|    Mestrado|  7.0|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|     23000.0|    Superior|  5.0|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|     11000.0|    Superior|  9.0|
|        7| Eduardo|  Extrem

In [ ]:
'''
Agora você vai aplicar seus conhecimentos de Pyspark para para as seguintes
transformações no DF:

1 - Renomear todas as colunas de forma que se mantenha os nomes, mas apenas com
letras minúsculas sem espaços, e/ou caracteres especiais

2 - Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas
diferentes

3 - Considerando que todos os funcionários tem um desconto de previdência complementar
de 10 % do salário Bruto crie e calcule os valores de uma nova coluna chamada
previdencia.

4 - Considerando que todos os funcionários nesse mês epecífico terão uma bonificacçao
de 20% sobre o salário bruto, crie e calcule os valores de uma nova coluna chamada
bonus

5 - Crie uma nova coluna com todos os nomes dos funcionários em letras maiúsculas

6 - Crie uma nova coluna com os nomes dos funcionários todos em letras minúsculas

7 - Crie um novo dataframe  chamado quant apensa com as colunas idade,
salário e notas

8 - calcule as principais medidas descritivas do df quant (describe)

9 - Ordene o df original por notas de forma crescente

10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''

In [ ]:
# 1) Renomear todas as colunas de forma que se mantenha os nomes, mas apenas com letras minúsculas sem espaços, e/ou caracteres especiais
df = df.withColumnRenamed('Matrícula','matricula')
df = df.withColumnRenamed('Nome','nome')
df = df.withColumnRenamed('Cidade','cidade')
df = df.withColumnRenamed('Estado','estado')
df = df.withColumnRenamed('País','pais')
df = df.withColumnRenamed('Idade','idade')
df = df.withColumnRenamed('Departamento','departamento')
df = df.withColumnRenamed('Cargo','cargo')
df = df.withColumnRenamed('Salário (R$)','salario')
df = df.withColumnRenamed('Escolaridade','escolaridade')
df = df.withColumnRenamed('Nota ','nota')
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|  25000|    Superior|   8|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|  28000|         MBA|   9|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|  40000|    Mestrado|   7|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|  23000|    Superior|   5|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|  11000|    Superior|   9|
|        7| Eduardo|  Extrema|    MG|Brasil|   30|   Marketing|    Gerente|  12000

In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro01 = df.filter((df['idade']>=30)&(df['nota']>=4.0))
filtro01.show()

+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|matricula|    nome| cidade|estado|  pais|idade|departamento|   cargo|salario|escolaridade|nota|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|        1|   Lucas|Atibaia|    SP|Brasil|   35|     Compras| Gerente|  25000|    Superior|   8|
|        3|   Luiza| Santos|    SP|Brasil|   38|    Finanças| Gerente|  28000|         MBA|   9|
|        4|Fernando|Atibaia|    SP|Brasil|   36|   Marketing| Diretor|  40000|    Mestrado|   7|
|        7| Eduardo|Extrema|    MG|Brasil|   30|   Marketing| Gerente|  12000|         MBA|   4|
|       10|   Maria|Extrema|    MG|Brasil|   40|    Produção|Analista|  12000|         MBA|   7|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro02 = df.filter((df['idade']>=35)&(df['salario']>12000))
filtro02.show()

+---------+--------+-------+------+------+-----+------------+-------+-------+------------+----+
|matricula|    nome| cidade|estado|  pais|idade|departamento|  cargo|salario|escolaridade|nota|
+---------+--------+-------+------+------+-----+------------+-------+-------+------------+----+
|        1|   Lucas|Atibaia|    SP|Brasil|   35|     Compras|Gerente|  25000|    Superior|   8|
|        3|   Luiza| Santos|    SP|Brasil|   38|    Finanças|Gerente|  28000|         MBA|   9|
|        4|Fernando|Atibaia|    SP|Brasil|   36|   Marketing|Diretor|  40000|    Mestrado|   7|
+---------+--------+-------+------+------+-----+------------+-------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro03 = df.filter((df['idade']>=35)&(df['idade']<=40))
filtro03.show()

+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|matricula|    nome| cidade|estado|  pais|idade|departamento|   cargo|salario|escolaridade|nota|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|        1|   Lucas|Atibaia|    SP|Brasil|   35|     Compras| Gerente|  25000|    Superior|   8|
|        3|   Luiza| Santos|    SP|Brasil|   38|    Finanças| Gerente|  28000|         MBA|   9|
|        4|Fernando|Atibaia|    SP|Brasil|   36|   Marketing| Diretor|  40000|    Mestrado|   7|
|       10|   Maria|Extrema|    MG|Brasil|   40|    Produção|Analista|  12000|         MBA|   7|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro04 = df.filter((df['idade']>=40)|(df['salario']>=25000))
filtro04.show()

+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|matricula|    nome| cidade|estado|  pais|idade|departamento|   cargo|salario|escolaridade|nota|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+
|        1|   Lucas|Atibaia|    SP|Brasil|   35|     Compras| Gerente|  25000|    Superior|   8|
|        3|   Luiza| Santos|    SP|Brasil|   38|    Finanças| Gerente|  28000|         MBA|   9|
|        4|Fernando|Atibaia|    SP|Brasil|   36|   Marketing| Diretor|  40000|    Mestrado|   7|
|       10|   Maria|Extrema|    MG|Brasil|   40|    Produção|Analista|  12000|         MBA|   7|
+---------+--------+-------+------+------+-----+------------+--------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro05 = df.filter(~(df['cargo'] == 'Analista'))
filtro05.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|  25000|    Superior|   8|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|  28000|         MBA|   9|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|  40000|    Mestrado|   7|
|        7| Eduardo|  Extrema|    MG|Brasil|   30|   Marketing|    Gerente|  12000|         MBA|   4|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro06 = df.filter(col('cidade').startswith('S'))
filtro06.show()

+---------+-----+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula| nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+-----+---------+------+------+-----+------------+-----------+-------+------------+----+
|        2|  Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        3|Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|  28000|         MBA|   9|
+---------+-----+---------+------+------+-----+------------+-----------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro07 = df.filter(~(df['departamento'] == 'Marketing'))
filtro07.show()

+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|   nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        1|  Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|  25000|    Superior|   8|
|        2|    Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        3|  Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|  28000|         MBA|   9|
|        5| Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|  23000|    Superior|   5|
|        6|Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|  11000|    Superior|   9|
|        8|  Ester|  Itapeva|    MG|Brasil|   29|     Compras|   Analista|  10000|    Superior|   2|
|       10|  Maria|  Extrema|    MG|Brasil|   40|    Produção|   Analista|  12000|         

In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro08 = df.filter(~(df['departamento'] == 'Marketing')&(df['salario']<=12000))
filtro08.show()

+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|   nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        2|    Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        6|Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|  11000|    Superior|   9|
|        8|  Ester|  Itapeva|    MG|Brasil|   29|     Compras|   Analista|  10000|    Superior|   2|
|       10|  Maria|  Extrema|    MG|Brasil|   40|    Produção|   Analista|  12000|         MBA|   7|
+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro09 = df.filter(col('cidade').startswith('S')|(df['cargo']=='Gerente'))
filtro09.show()

+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|   nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        1|  Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|  25000|    Superior|   8|
|        2|    Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        3|  Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|  28000|         MBA|   9|
|        7|Eduardo|  Extrema|    MG|Brasil|   30|   Marketing|    Gerente|  12000|         MBA|   4|
+---------+-------+---------+------+------+-----+------------+-----------+-------+------------+----+



In [ ]:
# 2) Fazer livremente no mínimo 10 filtros que tragam recortes a partir de colunas diferentes
filtro10 = df.filter(~(col('nome').startswith('L')))
filtro10.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|  12000|    Superior|   6|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|  40000|    Mestrado|   7|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|  23000|    Superior|   5|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|  11000|    Superior|   9|
|        7| Eduardo|  Extrema|    MG|Brasil|   30|   Marketing|    Gerente|  12000|         MBA|   4|
|        8|   Ester|  Itapeva|    MG|Brasil|   29|     Compras|   Analista|  10000|    Superior|   2|
|        9|   Pedro|  Extrema|    MG|Brasil|   30|   Marketing|   Analista|  13000

In [ ]:
#3) Considerando que todos os funcionários tem um desconto de previdência complementar de 10 % crie uma nova coluna chamada previdencia.
df = df.withColumn('previdencia',df['salario']*0.1)
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|previdencia|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|25000.0|    Superior| 8.0|     2500.0|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|12000.0|    Superior| 6.0|     1200.0|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|28000.0|         MBA| 9.0|     2800.0|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|40000.0|    Mestrado| 7.0|     4000.0|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|23000.0|    Superior| 5.0|     2300.0|
|        6| Douglas| Bragança|    SP|Brasil|   29|    Finanças|   Analista|11000.0|    S

In [ ]:
#4) Considerando que todos os funcionários tem um bônus complementar de 20 % crie uma nova coluna chamada bonus.
df = df.withColumn('bonus',df['salario']*0.2)
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|previdencia| bonus|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|25000.0|    Superior| 8.0|     2500.0|5000.0|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|12000.0|    Superior| 6.0|     1200.0|2400.0|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|28000.0|         MBA| 9.0|     2800.0|5600.0|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|40000.0|    Mestrado| 7.0|     4000.0|8000.0|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|23000.0|    Superior| 5.0|     2300.0|4600.0|
|        6| Douglas| Bragança|  

In [ ]:
# 5) Crie uma nova coluna com todos os nomes dos funcionários em letras minúsculas
df = df.withColumn('nomes_min',lower(col('nome')))
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+---------+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|previdencia| bonus|nomes_min|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+---------+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|25000.0|    Superior| 8.0|     2500.0|5000.0|    lucas|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|12000.0|    Superior| 6.0|     1200.0|2400.0|      ana|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|28000.0|         MBA| 9.0|     2800.0|5600.0|    luiza|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|40000.0|    Mestrado| 7.0|     4000.0|8000.0| fernando|
|        5|  Sandra|  Atibaia|    SP|Brasil|   28|    Produção|   Analista|23000.0|

In [ ]:
#6 - Crie uma nova coluna com os nomes dos funcionários todos em letras maiúsculas
df = df.withColumn('nomes_max',upper(col('nome')))
df.show()

+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+---------+---------+
|matricula|    nome|   cidade|estado|  pais|idade|departamento|      cargo|salario|escolaridade|nota|previdencia| bonus|nomes_min|nomes_max|
+---------+--------+---------+------+------+-----+------------+-----------+-------+------------+----+-----------+------+---------+---------+
|        1|   Lucas|  Atibaia|    SP|Brasil|   35|     Compras|    Gerente|25000.0|    Superior| 8.0|     2500.0|5000.0|    lucas|    LUCAS|
|        2|     Ana|São Paulo|    SP|Brasil|   29|      Vendas|Coordenador|12000.0|    Superior| 6.0|     1200.0|2400.0|      ana|      ANA|
|        3|   Luiza|   Santos|    SP|Brasil|   38|    Finanças|    Gerente|28000.0|         MBA| 9.0|     2800.0|5600.0|    luiza|    LUIZA|
|        4|Fernando|  Atibaia|    SP|Brasil|   36|   Marketing|    Diretor|40000.0|    Mestrado| 7.0|     4000.0|8000.0| fernando| FERNANDO|
|        5|  

In [ ]:
# 7) Crie um novo dataframe  chamado quant apensa com as colunas idade, salário e notas
quant = df.select('idade','salario','nota')
quant.show()

+-----+-------+----+
|idade|salario|nota|
+-----+-------+----+
|   35|25000.0| 8.0|
|   29|12000.0| 6.0|
|   38|28000.0| 9.0|
|   36|40000.0| 7.0|
|   28|23000.0| 5.0|
|   29|11000.0| 9.0|
|   30|12000.0| 4.0|
|   29|10000.0| 2.0|
|   30|13000.0| 1.0|
|   40|12000.0| 7.0|
+-----+-------+----+



In [ ]:
# 8) Calcule as principais medidas descritivas do df quant (describe)
quant.describe().show()

+-------+-----------------+------------------+-----------------+
|summary|            idade|           salario|             nota|
+-------+-----------------+------------------+-----------------+
|  count|               10|                10|               10|
|   mean|             32.4|           18600.0|              5.8|
| stddev|4.402019738458447|10002.221975363496|2.780887148615228|
|    min|               28|           10000.0|              1.0|
|    max|               40|           40000.0|              9.0|
+-------+-----------------+------------------+-----------------+



In [ ]:
# 9) Ordene o df original por notas de forma crescente
df.orderBy('nome')

matricula,nome,cidade,estado,pais,idade,departamento,cargo,salario,escolaridade,nota,previdencia,bonus,nomes_min,nomes_max
2,Ana,São Paulo,SP,Brasil,29,Vendas,Coordenador,12000.0,Superior,6.0,1200.0,2400.0,ana,ANA
6,Douglas,Bragança,SP,Brasil,29,Finanças,Analista,11000.0,Superior,9.0,1100.0,2200.0,douglas,DOUGLAS
7,Eduardo,Extrema,MG,Brasil,30,Marketing,Gerente,12000.0,MBA,4.0,1200.0,2400.0,eduardo,EDUARDO
8,Ester,Itapeva,MG,Brasil,29,Compras,Analista,10000.0,Superior,2.0,1000.0,2000.0,ester,ESTER
4,Fernando,Atibaia,SP,Brasil,36,Marketing,Diretor,40000.0,Mestrado,7.0,4000.0,8000.0,fernando,FERNANDO
1,Lucas,Atibaia,SP,Brasil,35,Compras,Gerente,25000.0,Superior,8.0,2500.0,5000.0,lucas,LUCAS
3,Luiza,Santos,SP,Brasil,38,Finanças,Gerente,28000.0,MBA,9.0,2800.0,5600.0,luiza,LUIZA
10,Maria,Extrema,MG,Brasil,40,Produção,Analista,12000.0,MBA,7.0,1200.0,2400.0,maria,MARIA
9,Pedro,Extrema,MG,Brasil,30,Marketing,Analista,13000.0,Superior,1.0,1300.0,2600.0,pedro,PEDRO
5,Sandra,Atibaia,SP,Brasil,28,Produção,Analista,23000.0,Superior,5.0,2300.0,4600.0,sandra,SANDRA


In [ ]:
'''
10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''
df.groupBy('cargo').agg({'cargo':'count'})

cargo,count(cargo)
Gerente,3
Coordenador,1
Diretor,1
Analista,5


In [ ]:
'''
10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''
df.groupBy('cidade').agg({'cidade':'count'})

cidade,count(cidade)
Santos,1
Extrema,3
São Paulo,1
Atibaia,3
Bragança,1
Itapeva,1


In [ ]:
'''
10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''
df.groupBy('estado').agg({'estado':'count'})

estado,count(estado)
SP,6
MG,4


In [ ]:
'''
10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''
df.groupBy('escolaridade').agg({'escolaridade':'count'})

escolaridade,count(escolaridade)
Superior,6
Mestrado,1
MBA,3


In [ ]:
'''
10 - faça as contagens de frequências, usando groupby para as seguintes colunas:
a) Cargo
b) Cidade
c) Estado
d) Escolaridade
e) Departamento
'''
df.groupBy('departamento').agg({'departamento':'count'})

departamento,count(departamento)
Vendas,1
Finanças,2
Marketing,3
Compras,2
Produção,2
